# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [179]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np
import re

def generate_energy_df():
    energy_df=pd.read_excel('assets\\Energy Indicators.xls')
    replaces_for_country={
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }
    # YOUR CODE HERE load dataframe and remove header and footer
    energy_df=energy_df.iloc[17:245, 2:]
    # YOUR CODE HERE transform data
    energy_df=energy_df.rename(columns={'Unnamed: 2':'Country Names', 'Unnamed: 3':'Energy Supply', 'Unnamed: 4':'Energy Supply per Capita', 'Unnamed: 5':'% Renewable'})
    energy_df = energy_df.replace('...', np.nan)
    # YOUR CODE HERE transform Country column
    energy_df['Energy Supply']=energy_df['Energy Supply']*10**6
    #Delete () and digits in country names
    energy_df['Country Names'] = energy_df['Country Names'].str.replace(r'\s*\(.*?\)', '', regex=True)
    energy_df['Country Names'] = energy_df['Country Names'].str.replace(r'\d', '', regex=True)
    energy_df['Country Names']=energy_df['Country Names'].replace(replaces_for_country)
    energy_df=energy_df.set_index('Country Names')
    
    return energy_df.head()

def get_GDP_df():
    def is_number(s: str) -> bool:
        """
        Check if a string is a number.

        Args:
            s (str): The string to check.

        Returns:
            A boolean indicating if the string is a number.False
        """
        try:
            int(s)  
            return True
        except ValueError:
            return 

    gdp_df= pd.read_csv('assets\\world_bank.csv')
    gdp_changes={
        'Data Source':'Country Names',
        'World Development Indicators':'Country Code',
        'Unnamed: 2':'Indicator Name',
        'Unnamed: 3':'Indicator Code',
        'Unnamed: 50':'2006',
        'Unnamed: 51':'2007',
        'Unnamed: 52':'2008',
        'Unnamed: 53':'2009',
        'Unnamed: 54':'2010',
        'Unnamed: 55':'2011',
        'Unnamed: 56':'2012',
        'Unnamed: 57':'2013',
        'Unnamed: 58':'2014',
        'Unnamed: 59':'2015'
    }
    country_changes={
        "Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }

    gdp_df=gdp_df.iloc[4:]
    gdp_df=gdp_df.rename(columns=gdp_changes)
    # YOUR CODE HERE load data, remove header, adjust column names
    # YOUR CODE HERE transform Country column 
    gdp_df['Country Names']=gdp_df['Country Names'].replace(country_changes)
    gdp_df = gdp_df.drop(gdp_df.columns[1:50], axis=1)
    gdp_df=gdp_df.set_index('Country Names')

    gdp_df.head()
    print(gdp_df.columns)
    print(gdp_df['2015'].head())
    return gdp_df

def get_scimago_df():
    # YOUR CODE HERE load data
    scimago_df=pd.read_excel('assets\\scimagojr-3.xlsx')
    scimago_df=scimago_df.iloc[:15]
    scimago_df=scimago_df.rename(columns={'Contry':'Country Names'})
    scimago_df=scimago_df.set_index('Country Names')
    return scimago_df

def answer_one():
    Data_inner_df= scimago_df.join(energy_df, how='inner').join(gdp_df, how='inner')
    #raise NotImplementedError()
    return Data_inner_df

In [172]:
gdp_df= pd.read_csv('assets\\world_bank.csv')
gdp_changes={
    'Data Source':'Country Names',
    'World Development Indicators':'Country Code',
    'Unnamed: 2':'Indicator Name',
    'Unnamed: 3':'Indicator Code',
    'Unnamed: 50':'2006',
    'Unnamed: 51':'2007',
    'Unnamed: 52':'2008',
    'Unnamed: 53':'2009',
    'Unnamed: 54':'2010',
    'Unnamed: 55':'2011',
    'Unnamed: 56':'2012',
    'Unnamed: 57':'2013',
    'Unnamed: 58':'2014',
    'Unnamed: 59':'2015'
}
country_changes={
    "Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}

gdp_df=gdp_df.iloc[4:]
gdp_df=gdp_df.rename(columns=gdp_changes)
# YOUR CODE HERE load data, remove header, adjust column names
# YOUR CODE HERE transform Country column 
gdp_df['Country Names']=gdp_df['Country Names'].replace(country_changes)
gdp_df = gdp_df.drop(gdp_df.columns[1:50], axis=1)
gdp_df=gdp_df.set_index('Country Names')

gdp_df.head()
print(gdp_df.columns)
print(gdp_df['2015'].head())

Index(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015'],
      dtype='object')
Country Names
Aruba                   NaN
Andorra                 NaN
Afghanistan    2.029415e+10
Angola         1.039106e+11
Albania        1.312082e+10
Name: 2015, dtype: float64


In [169]:
gdp_df = pd.read_csv('assets\\world_bank.csv')
print(gdp_df.columns)

Index(['Data Source', 'World Development Indicators', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56',
       'Unnamed: 57', 'Unnamed: 58', '

In [146]:
scimago_df=pd.read_excel('assets\\scimagojr-3.xlsx')
scimago_df=scimago_df.iloc[:15]
scimago_df=scimago_df.rename(columns={'Country':'Country Names'})
scimago_df=scimago_df.set_index('Country Names')
scimago_df.head(20)


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Country Names,,,,,,,
China,1,127050,126767,597237,411683,4.70,138
United States,2,96661,94747,792274,265436,8.20,230
Japan,3,30504,30287,223024,61554,7.31,134
United Kingdom,4,20944,20357,206091,37874,9.84,139
Russian Federation,5,18534,18301,34266,12422,1.85,57
Canada,6,17899,17620,215003,40930,12.01,149
Germany,7,17027,16831,140566,27426,8.26,126
India,8,15005,14841,128763,37209,8.58,115
France,9,13153,12973,130632,28601,9.93,114


In [145]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

energy_df=pd.read_excel('assets\\Energy Indicators.xls')
replaces_for_country={
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
energy_df=energy_df.iloc[17:245, 2:]
energy_df=energy_df.rename(columns={'Unnamed: 2':'Country Names', 'Unnamed: 3':'Energy Supply', 'Unnamed: 4':'Energy Supply per Capita', 'Unnamed: 5':'% Renewable'})
energy_df = energy_df.replace('...', np.nan)
energy_df['Energy Supply']=energy_df['Energy Supply']*10**6
#Delete () and digits in country names
energy_df['Country Names'] = energy_df['Country Names'].str.replace(r'\s*\(.*?\)', '', regex=True)
energy_df['Country Names'] = energy_df['Country Names'].str.replace(r'\d', '', regex=True)
energy_df['Country Names']=energy_df['Country Names'].replace(replaces_for_country)
energy_df=energy_df.set_index('Country Names')

print(energy_df)


                                            Energy Supply  \
Country Names                                               
Afghanistan                                  3.210000e+08   
Albania                                      1.020000e+08   
Algeria                                      1.959000e+09   
American Samoa                                        NaN   
Andorra                                      9.000000e+06   
Angola                                       6.420000e+08   
Anguilla                                     2.000000e+06   
Antigua and Barbuda                          8.000000e+06   
Argentina                                    3.378000e+09   
Armenia                                      1.430000e+08   
Aruba                                        1.200000e+07   
Australia                                    5.386000e+09   
Austria                                      1.391000e+09   
Azerbaijan                                   5.670000e+08   
Bahamas                 

C:\Users\Familia Mora\AppData\Local\Temp\ipykernel_2572\1287310630.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df = energy_df.replace('...', np.nan)


In [180]:
Data_inner_df= scimago_df.join(energy_df, how='inner').join(gdp_df, how='inner')
Data_inner_df.shape
#Data_inner_df.head()

(15, 20)

In [181]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [ ]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [182]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def answer_two():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [ ]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [ ]:
def answer_three():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [ ]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [ ]:
def answer_four():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [ ]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [ ]:
def answer_five():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [ ]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [ ]:
def answer_six():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [ ]:
def answer_seven():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [ ]:
def answer_eight():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [ ]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [ ]:
def answer_nine():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [ ]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [ ]:
def answer_ten():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [ ]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [ ]:
def answer_eleven():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [ ]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [ ]:
def answer_twelve():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"

In [ ]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [ ]:
def answer_thirteen():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [ ]:
# space for proffesor tests